In [ ]:
import gymnasium as gym

env = gym.make("room_env:RoomEnv-v1")
(obs, question), info = env.reset()
while True:
    (obs, question), reward, done, truncated, info = env.step("foo")
    if done:
        break

In [2]:
foo = env.step("foo")

In [3]:
len(foo)

5

In [ ]:
(observation, question), reward, done, truncated, info